# Polarization In Yeast
***
Reactions could not be completed from the slides (74 - 82)
***
## Setup the Environment
***

In [ ]:
import spatialpy

***
## Create the Polarization In Yeast Model
***

In [ ]:
def create_polarization_in_yeast(parameter_values=None):
    model = spatialpy.Model(name="Polarization_In_Yeast")

    model.CYTOPLASM = 'Cytoplasm'
    model.MEMBRANE = 'Membrane'

    # Domain
    domain = spatialpy.Domain.read_stochss_domain('Polarization_In_Yeast.smdl')
    model.add_domain(domain)

    model.staticDomain = True

    # Variables
    Cdc42c = spatialpy.Species(name="Cdc42c", diffusion_coefficient=10)
    Cdc42m = spatialpy.Species(name="Cdc42m", diffusion_coefficient=0.0053, restrict_to=[model.MEMBRANE])
    model.add_species([Cdc42c, Cdc42m])

    # Initial Conditions
    Cdc42c_ic = spatialpy.ScatterInitialCondition(species="Cdc42c", count=300, types=[model.CYTOPLASM])
    Cdc42m_ic = spatialpy.ScatterInitialCondition(species="Cdc42m", count=100, types=[model.MEMBRANE])
    model.add_initial_condition([Cdc42c_ic, Cdc42m_ic])

    # Parameters
    kon = spatialpy.Parameter(name="kon", expression="1.6666666667e-05")
    koff = spatialpy.Parameter(name="koff", expression="0.15")
    kfb = spatialpy.Parameter(name="kfb", expression="0.166666666666667")
    model.add_parameter([kon, koff, kfb])

    # Reactions
    r1 = spatialpy.Reaction(
        name="r1", rate="kon",
        reactants={'Cdc42c': 1}, products={'Cdc42m': 1}
    )
    r2 = spatialpy.Reaction(
        name="r2", rate="koff",
        reactants={'Cdc42m': 1}, products={'Cdc42c': 1}
    )
    r3 = spatialpy.Reaction(
        name="r3", rate="kfb", restrict_to=[model.MEMBRANE],
        reactants={'Cdc42c': 1, 'Cdc42m': 1}, products={'Cdc42m': 2}
    )
    model.add_reaction([r1, r2, r3])

    # Timespan
    tspan = spatialpy.TimeSpan.arange(10, t=5000, timestep_size=1)
    model.timespan(tspan)
    return model

### Instantiate the Model

In [ ]:
model = create_polarization_in_yeast()

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    kwargs = {
        # "number_of_trajectories":1,
        # "seed":None,
    }
    return kwargs

***
## Run the Simulation
***

In [ ]:
kwargs = configure_simulation()
results = model.run(**kwargs)

***
## Visualization
***

In [ ]:
results.plot_species('Cdc42c', animated=True, width=None, height=None)